In [ ]:
pip install transformers datasets

In [ ]:
from transformers import AutoModelForImageClassification
from datasets import load_dataset


In [ ]:
model = AutoModelForImageClassification.from_pretrained("facebook/deit-tiny-patch16-224")

In [ ]:
train_main = load_dataset("cifar10", split="train")
test = load_dataset("cifar10", split= "test")
split = train_main.train_test_split(test_size=0.1)
train = split['train']
val = split['test']

In [ ]:
from transformers import AutoImageProcessor

In [ ]:
pip install transformers[torch]

In [ ]:
import accelerate

In [ ]:
processor = AutoImageProcessor.from_pretrained("facebook/deit-tiny-patch16-224")

In [ ]:
from torchvision.transforms import (CenterCrop,
                                    Compose,
                                    Normalize,
                                    RandomHorizontalFlip,
                                    RandomResizedCrop,
                                    Resize,
                                    ToTensor)

image_mean, image_std = processor.image_mean, processor.image_std
size = processor.size["height"]

normalize = Normalize(mean=image_mean, std=image_std)
_train_transforms = Compose([
            RandomResizedCrop(size),
            RandomHorizontalFlip(),
            ToTensor(),
            normalize,])

_val_transforms = Compose([
            Resize(size),
            CenterCrop(size),
            ToTensor(),
            normalize,])

def train_transforms(examples):
    examples['pixel_values'] = [_train_transforms(image.convert("RGB")) for image in examples['img']]
    return examples

def val_transforms(examples):
    examples['pixel_values'] = [_val_transforms(image.convert("RGB")) for image in examples['img']]
    return examples

In [ ]:
train.set_transform(train_transforms)
val.set_transform(val_transforms)
test.set_transform(val_transforms)

In [ ]:
from transformers import TrainingArguments, Trainer

metric_name = "accuracy"

args = TrainingArguments(
    f"test-cifar-10",
    save_strategy="epoch",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=10,
    per_device_eval_batch_size=4,
    num_train_epochs=10,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    logging_dir='logs',
    remove_unused_columns=False,
)

In [ ]:
from torch.utils.data import DataLoader
import torch

def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

train_dataloader = DataLoader(train, collate_fn=collate_fn, batch_size=4)

In [ ]:
from sklearn.metrics import accuracy_score
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return dict(accuracy=accuracy_score(predictions, labels))


In [ ]:
import torch

trainer = Trainer(
    model,
    args,
    train_dataset=train,
    eval_dataset=val,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    tokenizer=processor,
)


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.550700,0.204900,0.935400
2,0.452600,0.164853,0.946400
3,0.423100,0.158358,0.950400
4,0.396000,0.158580,0.948800
5,0.388600,0.151114,0.955400
6,0.351300,0.145507,0.957600
7,0.334900,0.134586,0.962800
8,0.316100,0.128973,0.964200
9,0.287900,0.124801,0.965800
10,0.269700,0.122716,0.967000


TrainOutput(global_step=45000, training_loss=0.40487948811848956, metrics={'train_runtime': 3248.1619, 'train_samples_per_second': 138.54, 'train_steps_per_second': 13.854, 'total_flos': 2.3237042282496e+18, 'train_loss': 0.40487948811848956, 'epoch': 10.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.12271645665168762,
 'eval_accuracy': 0.967,
 'eval_runtime': 46.3411,
 'eval_samples_per_second': 107.896,
 'eval_steps_per_second': 26.974,
 'epoch': 10.0}

In [ ]:
torch.save(model,'deit_tiny_10.pt')

In [ ]:
from google.colab import files
files.download('deit_tiny_10.pt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.562300,0.211401,0.927400
2,0.477800,0.165889,0.945600
3,0.409800,0.157482,0.948400
4,0.388700,0.149390,0.951200
5,0.377500,0.142511,0.958800
6,0.334400,0.141260,0.958400
7,0.316000,0.139086,0.961200
8,0.309200,0.127307,0.964000
9,0.288100,0.125203,0.966200
10,0.288500,0.119803,0.968600


TrainOutput(global_step=45000, training_loss=0.40555804612901475, metrics={'train_runtime': 3358.5746, 'train_samples_per_second': 133.985, 'train_steps_per_second': 13.399, 'total_flos': 2.3237042282496e+18, 'train_loss': 0.40555804612901475, 'epoch': 10.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.11980312317609787,
 'eval_accuracy': 0.9686,
 'eval_runtime': 51.8887,
 'eval_samples_per_second': 96.36,
 'eval_steps_per_second': 24.09,
 'epoch': 10.0}

In [ ]:
quantized_model = torch.quantization.quantize_dynamic(
    model, {torch.nn.Linear}, dtype=torch.qint8)
print(quantized_model)

ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 192, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): DynamicQuantizedLinear(in_features=192, out_features=192, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
              (key): DynamicQuantizedLinear(in_features=192, out_features=192, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
              (value): DynamicQuantizedLinear(in_features=192, out_features=192, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): DynamicQuantizedLinear(in_features=192, out_fea

In [ ]:
import os
def print_size_of_model(model):
    torch.save(model.state_dict(), "temp.p")
    print('Size (MB):', os.path.getsize("temp.p")/1e6)
    os.remove('temp.p')

print_size_of_model(model)
print_size_of_model(quantized_model)

Size (MB): 22.936702
Size (MB): 6.492388


In [ ]:
torch.save(model, "deit_t_model.pth")

In [ ]:
from google.colab import files
files.download('deit_t_model.pth')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>